# Model Building for Hourly/Daily Predictions

    - labels are created separately (see 04-label-generation.ipynb)
    - labels are for alarms generated in the next 5 minutes, next 1 hour and next 1 day
    - data points can be at 5 minutes interval, hourly level and day lavel

In [ ]:
import datetime
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import pickle

import optuna
import lightgbm as lgb

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn import preprocessing


In [ ]:
with open('inverter-data-v03.pkl', 'rb') as handle:
    all_data = pickle.load(handle)
    
print(all_data.shape)
all_data.dropna(inplace=True)
all_data.shape

In [ ]:
all_data.columns

In [ ]:
# filter for hourly data
all_data = all_data[all_data.date.dt.minute==0].copy()

# filter for daily data
all_data = all_data[all_data.date.dt.hour==6]

In [ ]:
all_data.drop(columns=['label', 'label_1h', 'inverter'], inplace=True)

In [ ]:
categoricals = ['hour', "day", "dayofweek", "weekofyear", "month"]
for cat in categoricals:
    all_data[cat] = all_data[cat].astype('category')

In [ ]:
label_col = 'label_24h'
all_data[label_col].value_counts()

In [ ]:
total = all_data[label_col].value_counts()
print(f"Class ratio: {100 * total[1] / (total[0] + total[1]):.3f} %")

In [ ]:
split_type = 'random'
train_days, eval_days = 60, 60
dmin, dmax = all_data.date.min(), all_data.date.max()
train_end = dmax - pd.Timedelta(eval_days, 'D')
train_start = train_end - pd.Timedelta(train_days, 'D')
print(dmin, dmax, train_start, train_end)

if split_type == 'random':
    # random split
    train, test = train_test_split(all_data, train_size=0.8, random_state=100)

else:
    # train, test = all_data[all_data.date < train_cutoff], all_data[all_data.date >= train_cutoff]
    train, test = all_data[(all_data.date >= train_start) & (all_data.date < train_end)], all_data[all_data.date >= train_end]

train, valid = train_test_split(train, train_size=0.8, random_state=100)
print(train.shape, valid.shape, test.shape)

train[label_col].value_counts(True)

In [ ]:
test[label_col].value_counts(True)

In [ ]:
all_data['yearmo'] = all_data['date'].apply(lambda x: f"{x.year}{x.month:02d}")
all_data['yearmo'].value_counts()

In [ ]:
dft = all_data[['yearmo', label_col]].groupby(['yearmo']).agg(np.mean).reset_index()
dft.plot(x='yearmo')

In [ ]:
all_data.drop(columns=['yearmo'], inplace=True)

## Hyperparameter Tuning for LightGBM Model

In [ ]:
class Objective(object):
    def __init__(self, df_train, df_valid, categoricals, fixed_params, param_set={}, verbose_eval=50):
        self.categoricals = categoricals
        self.fixed_params = fixed_params
        self.param_set = param_set
        self.verbose_eval = verbose_eval
        self.dtrain = lgb.Dataset(
            df_train.drop([label_col], axis=1),
            label = df_train[label_col],
            categorical_feature=self.categoricals,
            free_raw_data=False
        )
        self.dvalid = lgb.Dataset(
            df_valid.drop([label_col], axis=1),
            label = df_valid[label_col],
            categorical_feature=self.categoricals,
            reference=self.dtrain,
            free_raw_data=False
        )
        self.default_ranges = {
            "num_leaves":(2, 256),
            "min_data_in_leaf":(5, 100),
            "learning_rate":(1e-3, 1e-1),
            "feature_fraction":(0.4, 1.0),
            "bagging_freq":(1, 7),
            "bagging_fraction":(0.4, 1.0)
        }
        
    def get_params(self, trial):
        param_funcs = {
            "num_leaves":trial.suggest_int,
            "min_data_in_leaf":trial.suggest_int,
            "learning_rate":trial.suggest_loguniform,
            "feature_fraction":trial.suggest_float,
            "bagging_freq":trial.suggest_int,
            "bagging_fraction":trial.suggest_float
        }
        params = {}
        for param, rng in self.param_set.items():
            if rng is None:
                default_rng = self.default_ranges[param]
                params[param] = param_funcs[param](param, default_rng[0], default_rng[1])
            else:
                params[param] = param_funcs[param](param, rng[0], rng[1])

        params.update(self.fixed_params)
        return params
    
    def __call__(self, trial):
        params = self.get_params(trial)
        bst = lgb.train(
            params,
            self.dtrain,
            valid_sets=[self.dvalid],
            verbose_eval=self.verbose_eval
        )
        # get best value of objective
        valid_0 = bst.best_score['valid_0']
        score = valid_0[list(valid_0)[0]]
        
        trial.set_user_attr('best_iteration', bst.best_iteration)
        trial.set_user_attr('features', self.dtrain.feature_name)
        trial.set_user_attr('importance', bst.feature_importance().tolist())
        
        return score

class EarlyStoppingExceeded(optuna.exceptions.OptunaError):
    pass

class EarlyStoppingCallback(object):
    # from https://github.com/optuna/optuna/issues/1001#issuecomment-596478792
    
    def __init__(self, early_stopping_rounds, min_delta):
        self.early_stopping_rounds = early_stopping_rounds
        self.min_delta = min_delta
        self.early_stopping_count = 0
        self.best_score = None
    
    def __call__(self, study, trial):
        if self.best_score == None:
            self.best_score = study.best_value

        if study.best_value < self.best_score - self.min_delta:
            self.best_score = study.best_value
            self.early_stopping_count = 0
        else:
            if self.early_stopping_count > self.early_stopping_rounds:
                self.early_stopping_count = 0
                best_score = None
                raise EarlyStoppingExceeded()
            else:
                self.early_stopping_count += 1
        return
    

def tune_model(df_train, df_valid, categoricals, fixed_params, param_set, n_trials=50, verbose_eval=50, show_progress=True, early_stop_callback=None, tpe_mode="independent"):
    multivariate_flag = True if tpe_mode == "multivariate" else False
    sampler = optuna.samplers.TPESampler(multivariate=multivariate_flag)
    study = optuna.create_study(sampler=sampler)
    callbacks = None
    if early_stop_callback is not None:
        callbacks = [early_stop_callback]
    else:
        callbacks = []
    try:
        study.optimize(
            Objective(
                df_train=df_train,
                df_valid=df_valid,
                categoricals=categoricals,
                fixed_params=fixed_params,
                param_set = param_set,
                verbose_eval=verbose_eval
            ),
            n_trials=n_trials,
            show_progress_bar=show_progress,
            callbacks=callbacks
        )
    except EarlyStoppingExceeded:
        print(f'EarlyStopping Exceeded: No new best scores on iters {early_stop_callback.early_stopping_rounds}')
    return study



In [ ]:
obj_func = 'binary'
num_rounds = 1000
early_stopping_rounds = 50

print("Tune hyperparameters...")
param_set = {
    "num_leaves":None, 
    "min_data_in_leaf":None, 
    "learning_rate":None, 
    "feature_fraction":None,
    "bagging_freq":None, 
    "bagging_fraction":None
}

fixed_params = {
    "objective":obj_func,
    "metric":[obj_func, "auc"],
    "num_rounds":num_rounds,
    "early_stopping_rounds":early_stopping_rounds,
    "first_metric_only":True,
    "force_row_wise":True,
    "feature_pre_filter":False,
    "verbose":1,
}

early_stopping = EarlyStoppingCallback(10, 0.001)

study = tune_model(
                    train.drop(columns=["date"]),
                    valid.drop(columns=["date"]),
                    categoricals, fixed_params, param_set, n_trials=100, verbose_eval=0,
                    show_progress=False, early_stop_callback=early_stopping,
                )

print("Saving best model parameters...")
best_params = {k: [v] for (k,v) in study.best_params.items()}


In [ ]:
with open('best_params_24h.pkl', 'wb') as handle:
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print('best parameters:', best_params)
num_rounds = study.best_trial.user_attrs["best_iteration"]

fixed_params["num_rounds"] = num_rounds
# fixed_params["early_stopping_rounds"] = 0
params = study.best_params.copy()

params.update(fixed_params)
del params["early_stopping_rounds"] # = 0        

In [ ]:
params['verbose'] = 1
params['metric'] = ['binary', 'auc']
# params['is_unbalance'] = True
params

In [ ]:
model = lgb.LGBMClassifier(boosting_type='gbdt', 
                           num_leaves=params['num_leaves'], 
                           min_data_in_leaf=params['min_data_in_leaf'],
                           learning_rate=params['learning_rate'],
                           feature_fraction=params['feature_fraction'],
                           bagging_freq=params['bagging_freq'],
                           bagging_fraction=params['bagging_fraction'],
                           objective='binary',
                           metric=params['metric'],
                           num_rounds=params['num_rounds'],
#                            is_unbalance=params['is_unbalance']
                          )
x_train, y_train = train.drop(columns=[label_col, "date"]), train[label_col]
x_val, y_val = valid.drop(columns=[label_col, "date"]), valid[label_col]

model.fit(X=x_train, y=y_train, 
          eval_set=[(x_val, y_val)],
          eval_names=['eval']
         )     

In [ ]:
x_test, y_test = test.drop(columns=[label_col, "date"]), test[label_col]

train_pred = model.predict_proba(x_train)
val_pred = model.predict_proba(x_val)
test_pred = model.predict_proba(x_test)
train_pred.shape, val_pred.shape, test_pred.shape

In [ ]:
model.feature_importances_, model.importance_type, model.classes_[1]

In [ ]:
train_auc = roc_auc_score(y_true=y_train, y_score=train_pred[:,1])
val_auc = roc_auc_score(y_true=y_val, y_score=val_pred[:,1])
test_auc = roc_auc_score(y_true=y_test, y_score=test_pred[:,1])
print(f"train-auc: {train_auc}, val-auc: {val_auc}, test-auc: {test_auc}")

In [ ]:
num = 10
feature_imp = pd.DataFrame({'Value':model.feature_importances_,
                            'Feature':train.drop(columns=[label_col, "date"]).columns})
plt.figure(figsize=(10, 5))
sns.set(font_scale = 1.5)
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", 
                                                    ascending=False)[0:num])
plt.title('Feature Importance')
plt.tight_layout()
plt.savefig('lgbm_importances-01.png')
plt.show()

## Precision-Recall Curve for the Test Data

In [ ]:
pos_label = 1
average_precision = average_precision_score(y_test, test_pred[:,1])
precision, recall, thresholds = precision_recall_curve(y_test, test_pred[:,1])
# disp = plot_precision_recall_curve(classifier, X_test, y_test)
# disp.ax_.set_title('2-class Precision-Recall curve: '
#                    'AP={0:0.2f}'.format(average_precision))
line_kwargs1 = {"drawstyle": "steps-post", 'label': 'precision'}
line_kwargs2 = {"drawstyle": "steps-post", 'label': 'threshold'}
# line_kwargs = {}
# line_kwargs["label"] = ('precision', 'threshold')

plt.figure(figsize=(10, 5))
sns.set(font_scale = 1)
fig, ax = plt.subplots()
ax.plot(recall, precision, **line_kwargs1)
ax.plot(recall[:-1], thresholds, **line_kwargs2)
info_pos_label = (f" (Positive label: {pos_label})"
                  if pos_label is not None else "")
xlabel = "Recall" + info_pos_label
ylabel = "Precision" + info_pos_label
ax.set(xlabel=xlabel, ylabel=ylabel, title=f"Average Precision = {average_precision:0.2f}")
ax.legend(loc="lower left")

## Precision-Recall Curve for the Validation Data

In [ ]:
pos_label = 1
average_precision = average_precision_score(y_val, val_pred[:,1])
precision, recall, thresholds = precision_recall_curve(y_val, val_pred[:,1])
line_kwargs1 = {"drawstyle": "steps-post", 'label': 'precision'}
line_kwargs2 = {"drawstyle": "steps-post", 'label': 'threshold'}

plt.figure(figsize=(10, 5))
sns.set(font_scale = 1)
fig, ax = plt.subplots()
ax.plot(recall, precision, **line_kwargs1)
ax.plot(recall[:-1], thresholds, **line_kwargs2)
info_pos_label = (f" (Positive label: {pos_label})"
                  if pos_label is not None else "")
xlabel = "Recall" + info_pos_label
ylabel = "Precision" + info_pos_label
ax.set(xlabel=xlabel, ylabel=ylabel, title=f"Average Precision = {average_precision:0.2f}")
ax.legend(loc="lower left")

In [ ]:
all_data['date'].value_counts()

In [ ]:
all_data[all_data['date']=='2020-01-30 12:25:00']

In [ ]:
all_data.date.min(), all_data.date.max()

In [ ]:
dfg = all_data.groupby('inverter')
for inv, df in dfg:
    print(inv)
    print(df)
    sys.exit()

In [ ]:
df2 = df[df.date.dt.minute==0]

In [ ]:
df2.label_1h.value_counts(True)

In [ ]:
df.label_1h.value_counts(True)

In [ ]:
df = all_data[all_data.date.dt.hour==6]
df.label_24h.value_counts(True)

In [ ]:
df.date.dt.hour.value_counts()

In [ ]:
df.date.min(), df.date.max()

In [ ]:
df.date.value_counts()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

features = ['power_max8640', 'power_max6048', 'power_max4032', 'power_max2016', 'power_max864', 'power_max576']

for col in features:
    df = all_data[[col, 'label_1h']]
    print(df.groupby('label_1h').agg(np.mean))

    data1 = df[df.label_1h==1][col]
    data2 = df[df.label_1h==0][col]

    density1 = gaussian_kde(data1)
    density2 = gaussian_kde(data2)

    xs = np.linspace(300, 500, 100)
    density1.covariance_factor = lambda : .25
    density1._compute_covariance()

    density2.covariance_factor = lambda : .25
    density2._compute_covariance()
    plt.plot(xs, density1(xs), label='positive')
    plt.plot(xs, density2(xs), label='negative')
    plt.title(f'Density plot: {col}')
    plt.legend()
    plt.show()

In [ ]:
features = ['power_std8640', 'power_mean8640']

for col in features:
    df = all_data[[col, 'label_1h']]
    print(df.groupby('label_1h').agg(np.mean))

    data1 = df[df.label_1h==1][col]
    data2 = df[df.label_1h==0][col]

    density1 = gaussian_kde(data1)
    density2 = gaussian_kde(data2)

    xs = np.linspace(0, 200, 200)
    density1.covariance_factor = lambda : .25
    density1._compute_covariance()

    density2.covariance_factor = lambda : .25
    density2._compute_covariance()
    plt.plot(xs, density1(xs), label='positive')
    plt.plot(xs, density2(xs), label='negative')
    plt.title(f'Density plot: {col}')
    plt.legend()
    plt.show()

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
train.shape

In [ ]:
train.columns

In [ ]:
# df = train.sample(n=100000, random_state=100)
# X, y = df.drop(columns=['label_1h', 'date']), df['label_1h']
label_col = 'label_24h'
X, y = train.drop(columns=[label_col, 'date']), train[label_col]

clf = IsolationForest(random_state=0).fit(X)

In [ ]:
train_pred = clf.predict(train.drop(columns=[label_col, 'date']))

In [ ]:
res = pd.DataFrame({'pred': train_pred, 'y': train[label_col]})
res['pred'] = res['pred'].map({1: 0, -1: 1})

In [ ]:
pd.crosstab(res['y'], res['pred'], rownames=['actual'], colnames=['predicted'])

In [ ]:
test_pred = clf.predict(test.drop(columns=[label_col, 'date']))
res = pd.DataFrame({'pred': test_pred, 'y': test[label_col]})
res['pred'] = res['pred'].map({1: 0, -1: 1})
pd.crosstab(res['y'], res['pred'], rownames=['actual'], colnames=['predicted'])

In [ ]:
print(classification_report(res['y'], res['pred'], target_names=['Normal', 'Faulty']))

In [ ]:
113/(113 + 1086)

## One Class SVM

In [ ]:
from sklearn.svm import OneClassSVM

*Hyper-parameters*

- kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’}
- degree of the poly kernel, 1, 2, 3, ...
- gamma: scale, auto or float
- coef0: independent term in kernel function, for 'poly' and 'sigmoid'
- nu: upper bound on the fraction of training errors and lower bound on the fraction of support vectors

In [ ]:
normalize = False

X, y = train.drop(columns=[label_col, 'date']), train[label_col]
X.describe()

In [ ]:
if normalize:
    scaler = preprocessing.StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    clf = OneClassSVM(gamma='auto').fit(X_scaled)
    train_pred = clf.predict(X_scaled)
    test_pred = clf.predict(scaler.transform(test.drop(columns=[label_col, 'date'])))
    test_score = clf.score_samples(scaler.transform(test.drop(columns=[label_col, 'date'])))

else:
    # clf = OneClassSVM(gamma='scale').fit(X)
    clf = OneClassSVM(gamma='auto').fit(X)
    train_pred = clf.predict(X)
    test_pred = clf.predict(test.drop(columns=[label_col, 'date']))
    test_score = clf.score_samples(test.drop(columns=[label_col, 'date']))
    
res_train = pd.DataFrame({'pred': train_pred, 'y': train[label_col]})
# 1-class SVM prediction in {-1, +1}, -1 means outlier
res_train['pred'] = res_train['pred'].map({1: 0, -1: 1})
pd.crosstab(res_train['y'], res_train['pred'], rownames=['actual'], colnames=['predicted'])

In [ ]:
res = pd.DataFrame({'pred': test_pred, 'y': test[label_col]})
res['pred'] = res['pred'].map({1: 0, -1: 1})
pd.crosstab(res['y'], res['pred'], rownames=['actual'], colnames=['predicted'])

In [ ]:
print(classification_report(res['y'], res['pred'], target_names=['Normal', 'Faulty']))

In [ ]:
test_score = clf.score_samples(test.drop(columns=[label_col, 'date']))

In [ ]:
pos_label = 1
average_precision = average_precision_score(test[label_col], test_score)
precision, recall, thresholds = precision_recall_curve(test[label_col], test_score)
line_kwargs1 = {"drawstyle": "steps-post", 'label': 'precision'}
line_kwargs2 = {"drawstyle": "steps-post", 'label': 'threshold'}

plt.figure(figsize=(10, 5))
sns.set(font_scale = 1)
fig, ax = plt.subplots()
ax.plot(recall, precision, **line_kwargs1)
ax.plot(recall[:-1], thresholds, **line_kwargs2)
info_pos_label = (f" (Positive label: {pos_label})"
                  if pos_label is not None else "")
xlabel = "Recall" + info_pos_label
ylabel = "Precision" + info_pos_label
ax.set(xlabel=xlabel, ylabel=ylabel, title=f"Average Precision = {average_precision:0.2f}")
ax.legend(loc="lower left")

In [ ]:
test_score.min(), test_score.max()

In [ ]:
y.describe()

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression

In [ ]:
train_score = clf.score_samples(X)
train_score.min(), train_score.max()

In [ ]:
train_score = np.expand_dims(train_score, -1)

In [ ]:
clf_lr = LogisticRegression(random_state=0).fit(train_score, y)

In [ ]:
calibrated_train_score = clf_lr.predict_proba(train_score)
calibrated_train_score.shape

In [ ]:
calibrated_train_score[:,1].min(), calibrated_train_score[:,1].max()

In [ ]:
test_score = np.expand_dims(test_score, -1)
calibrated_test_score = clf_lr.predict_proba(test_score)
calibrated_test_score.shape

In [ ]:
calibrated_test_score[:,1].min(), calibrated_test_score[:,1].max()

In [ ]:
# reliability diagram
fop, mpv = calibration_curve(test_y, calibrated_test_score[:,1], n_bins=10, normalize=True)

# plot perfectly calibrated
plt.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
plt.plot(mpv, fop, marker='.')
plt.show()

In [ ]:
# base_clf = OneClassSVM(gamma='auto')
# calibrated_clf = CalibratedClassifierCV(base_estimator=base_clf, cv=3)
# calibrated_clf.fit(X, y)

# calibrator = CalibratedClassifierCV(clf, cv='prefit')
# calibrator.fit(X, y)

In [ ]:
test_x, test_y = test.drop(columns=[label_col, 'date']), test[label_col]

# predict probabilities
probs = clf.decision_function(test_x)

# reliability diagram
fop, mpv = calibration_curve(test_y, probs, n_bins=10, normalize=True)

# plot perfectly calibrated
plt.plot([0, 1], [0, 1], linestyle='--')
# plot model reliability
plt.plot(mpv, fop, marker='.')
plt.show()

In [ ]:
dir(clf)

In [ ]:
# import optuna
# import sklearn
# from sklearn import datasets
# def objective(trial):
#     iris = sklearn.datasets.load_iris()
#     n_estimators = trial.suggest_int('n_estimators', 2, 20)
#     max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
#     clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
#     return sklearn.model_selection.cross_val_score(clf, iris.data, iris.target, 
#        n_jobs=-1, cv=3).mean()